In [1]:
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread
import numpy as np
import argparse
import imutils
import time
import dlib
import cv2
import os
import time
from grovepi import *
import math
from grove_rgb_lcd import *


In [2]:
setRGB(158,170,169)
setText("Advance Driving Monitoring System")
def alarm(msg):
    global alarm_status
    global alarm_status2
    global saying

    if alarm_status2:
        print('call')
        saying = True
        s = 'espeak "' + msg + '"'
        os.system(s)
        saying = False

In [3]:
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    C = dist.euclidean(eye[0], eye[3])

    ear = (A + B) / (2.0 * C)

    return ear

def final_ear(shape):
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

    leftEye = shape[lStart:lEnd]
    rightEye = shape[rStart:rEnd]

    leftEAR = eye_aspect_ratio(leftEye)
    rightEAR = eye_aspect_ratio(rightEye)

    ear = (leftEAR + rightEAR) / 2.0
    return (ear, leftEye, rightEye)

def lip_distance(shape):
    top_lip = shape[50:53]
    top_lip = np.concatenate((top_lip, shape[61:64]))

    low_lip = shape[56:59]
    low_lip = np.concatenate((low_lip, shape[65:68]))

    top_mean = np.mean(top_lip, axis=0)
    low_mean = np.mean(low_lip, axis=0)

    distance = abs(top_mean[1] - low_mean[1])
    return distance

In [4]:
EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 30
YAWN_THRESH = 20
alarm_status = False
alarm_status2 = False
saying = False
COUNTER = 0
bcount=0

In [5]:
print("-> Loading the predictor and detector...")
#detector = dlib.get_frontal_face_detector()
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")    #Faster but less accurate
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

-> Loading the predictor and detector...


In [6]:
print("-> Starting Video Stream")
vs = VideoStream(src=0).start()
#vs= VideoStream(usePiCamera=True).start()       //For Raspberry Pi
time.sleep(1.0)

-> Starting Video Stream


In [7]:
while True:

    frame = vs.read()
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #rects = detector(gray, 0)
    rects = detector.detectMultiScale(gray, scaleFactor=1.1, 
        minNeighbors=5, minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE)

    #for rect in rects:
    for (x, y, w, h) in rects:
        rect = dlib.rectangle(int(x), int(y), int(x + w),int(y + h))
        
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        eye = final_ear(shape)
        ear = eye[0]
        leftEye = eye [1]
        rightEye = eye[2]

        distance = lip_distance(shape)

        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

        lip = shape[48:60]
        cv2.drawContours(frame, [lip], -1, (0, 255, 0), 1)

        if ear < EYE_AR_THRESH:
            COUNTER += 1

            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                if alarm_status == False:
                    alarm_status = True
                    bcount=1
                    t = Thread(target=alarm, args=('wake up sir',))
                    t.deamon = True
                    t.start()
                    #**************************************************************
                    buzzer_pin = 3       #Port for buzzer
                    button = 2           #Port for Button

                    pinMode(buzzer_pin,"OUTPUT")	# Assign mode for buzzer as output
                    pinMode(button,"INPUT")		# Assign mode for Button as input
                    while True:
                        try:
                            button_status= digitalRead(button)	#Read the Button status 
                            
                            if button_status == 1:	#If the Button is in HIGH position, run the program
                                digitalWrite(buzzer_pin,0)
                                break

                                # print "\tBuzzing"			
                            else:		#If Button is in Off position, print "Off" on the screen
                                digitalWrite(buzzer_pin,1)
                                
                                if(bcount==1):
                                    setRGB(255,0,0)
                                    setText("Wake Up sir,    Press the Button")
                                bcount=bcount+1
    
                        except KeyboardInterrupt:	# Stop the buzzer before stopping
                            #digitalWrite(buzzer_pin,1)
                            setText("Entered in the exception area")
                            break
                        except (IOError,TypeError) as e:
                            print("Error")
                    setRGB(20,255,0)
                    setText("Drive Safe")
                    bcount=0
                    #*******************************************************

#                 cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        else:
            COUNTER = 0
            alarm_status = False

        if (distance > YAWN_THRESH):
                setRGB(255,95,0)
                #**** system is slipping for 2 sec to display ************
                setText("Yawn Alert,      Take some fresh air")
                time.sleep(2)
                setRGB(20,255,0)
                setText("Drive Safe")
                if alarm_status2 == False and saying == False:
                    alarm_status2 = True
                    t = Thread(target=alarm, args=('take some fresh air sir',))
                    t.deamon = True
                    t.start()
        else:
            alarm_status2 = False
        #this text is for frame view text(right top)
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "YAWN: {:.2f}".format(distance), (300, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)


    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):
        break

cv2.destroyAllWindows()
vs.stop()

call
call
call


KeyboardInterrupt: 